# Introduction

The goal of this project is to develop a library for performing automatic differentiation (AD), which we will call ```LazyDiff```.
AD is a set of techniques to numerically evaluate the derivative of a function motivated by deficiencies in classical methods. Numerical approximation is often not accurate enough and can be computationally expensive and symbolic differentiation tends to lead to inefficient code and faces the difficulty of converting a computer program into a single expression. Both classical methods have problems with calculating higher derivatives, where the complexity and errors increase, and are slow when it comes to computing the partial derivatives of a function with respect to many inputs. Automatic differentiation endeavors to solve all of these problems, and has been used in neural networks for back propagation and weight adjustment based on the loss function, and in scientific computing where it is often difficult to analytically compute the derivatives.





# How to Use LazyDiff

## How to Install

If you want to install the package only for the current virtual environment, run the following commands to create and activate a new virtual env:

- Move to the desired working directory
- `virtualenv env`
- `source env/bin/activate`

If you are cloning our repo and installing lazydiff that way, you can run the following to install the dependencies:
- `pip install -r requirements.txt`

Otherwise, you can execute following command to install LazyDiff and its dependencies:
 - `pip install git+https://github.com/CS207-Project-Group-7/cs207-FinalProject.git@code`

In [0]:
# REMEMBER TO ACTIVATE YOUR VIRTUALENV FIRST
# Install lazydiff for demos
!pip install git+https://github.com/CS207-Project-Group-7/cs207-FinalProject.git@code

  Cloning https://github.com/CS207-Project-Group-7/cs207-FinalProject.git (to revision code) to /tmp/pip-req-build-p2clugye
Branch 'code' set up to track remote branch 'code' from 'origin'.
Switched to a new branch 'code'
  Running setup.py bdist_wheel for lazydiff ... - done
  Stored in directory: /tmp/pip-ephem-wheel-cache-imalahe5/wheels/38/b0/a1/23fd2707fc03de253c749cba036aab7b1e2cf887a06df4abf4
Successfully built lazydiff


## Basic Demo
 
 ### Variable Objects and Evaluating Functions 
 
 The values of variables are stored inside `vars` objects as `Var`, which is used as function inputs. Custom functions from `ops` module need to be used to evaluate the functions, which support all the standard math functions from math and numpy libraries. 
 
 Function values can be evaluated by simply wrapping the variables with the corresponding functions and accessing the values using `.val`
 
Below we show the examples for basic operations and some of our elementary functions. 

We evaluate $f(x) = x^2$ and $f(y) = -y$ for variables $x=4$ and $y=2$. We also show how we can compound the functions by evaluating $f(z) = 5*z$ where $z = f(x) + f(y)$, $z = f(x) - f(y)$ and $z = \frac{f(x)}{f(y)}$, respectively.
 
 Also, we evaluate $f(a) = sin(a)$ for $a = 0$,  $f(b) = arcsin(b)$ for $b=0$, and $f(c) = \frac{1}{1 + e^{-{c}}} + log_2(c)+ \sqrt{c}$  for $c = 1$.

In [0]:
from lazydiff.vars import Var
from lazydiff import ops
import math

#part 1
# creating variable x = 4 and y = 2
x = Var(4)
y = Var(2)

# constructing and evaluating f(x) = x^2 and f(y) = sin(y)
fx = x**2
fy = -y
# compound function f(z)_1 = 5*z where z = fx + fy
fz_1 = 5*(fx + fy)
# compound function f(z)_2 = 5*z where z = fx - fy
fz_2 = 5*(fx - fy)
# compound function f(z)_3 = 5*z where z = fx / fy
fz_3 = 5*(fx / fy)

# accessing the evaluated values
# output should print out 16, -2, 70, 90 and -40
print("Evaluating f(x) = x^2 where x = 4 : {}".format(fx.val))
print("Evaluating f(y) = -y where y = 2 : {}".format(fy.val))
print("Evaluating f(z) = 5*z where z = f(x) + f(y) : {}".format(fz_1.val))
print("Evaluating f(z) = 5*z where z = f(x) - f(y) : {}".format(fz_2.val))
print("Evaluating f(z) = 5*z where z = f(x) / f(y) : {}".format(fz_3.val))

#part 2
# creating variable a = 0, b = 0 and c = 1
a = Var(0)
b = Var(0)
c = Var(1)

# constructing and evaluating f(a) = sin(a), f(b) = arcsin(b) and f(c) = 1/(1+e^(-c)) + log2(c) + c^(1/2)
fa = ops.sin(a)
fb = ops.arcsin(b)
fc = ops.logistic(c) + ops.log(c, 2) + ops.sqrt(c)
# accessing the evaluated values
# output should print out 0, 0 and 1.73
print("Evaluating f(a) = sin(a) where a = 0 : {}".format(fa.val))
print("Evaluating f(b) = arcsin(b) where b = 0 : {}".format(fb.val))
print("Evaluating f(c) = 1/(1+e^(-c)) + log2(c) + c^(1/2) where c = 1 : {}".format(fc.val))




Evaluating f(x) = x^2 where x = 4 : 16.0
Evaluating f(y) = -y where y = 2 : -2.0
Evaluating f(z) = 5*z where z = f(x) + f(y) : 70.0
Evaluating f(z) = 5*z where z = f(x) - f(y) : 90.0
Evaluating f(z) = 5*z where z = f(x) / f(y) : -40.0
Evaluating f(a) = sin(a) where a = 0 : 0.0
Evaluating f(b) = arcsin(b) where b = 0 : 0.0
Evaluating f(c) = 1/(1+e^(-c)) + log2(c) + c^(1/2) where c = 1 : 1.7310585786300048


### Evaluating Function Gradients

#### Forward Mode

As each function is evaluated, the corresponding gradient is also stored in the new output variable. Therefore, each of the output variable contains the gradients from all the previously evaluated functions. This allows us to access the gradient with respect to any of the variables previously computed. For instance, for $f(z)$ from the above example we can compute the gradient with respect to $z$ or $x$ depending on what we pass as input to the gradient method. This also makes it necessary to store the variables of interest somewhere in the program and then use the variable for which we want to evaluate the gradient of as input to the gradient method of the function. 

Below we show an example of how to access gradients of $f(x)$, $f(y)$, $f(z)$ and $f(a)$, $f(b)$, $f(c)$using `.grad(variable)` method via forward mode.

In [0]:
# accessing gradients
#part 1
# gradient of f(x) = x^2 w.r.t x is 8
x.forward()
print("Gradient of f(x) = x^2 w.r.t x is {}".format(fx.grad(x)))
# gradient of f(y) = -y w.r.t y is -1
y.forward()
print("Gradient of f(y) = -y w.r.t y is {}".format(fy.grad(y)))

# gradient of fz_1 w.r.t x is d5(x^2-y)/dx = 10x = 40
print("Gradients of f(z) = 5*z where z = f(x) + f(y) w.r.t x is {}".format(fz_1.grad(x)))
# gradient of fz_2 w.r.t x is d5(x^2+y)/dx = 10x = 40
print("Gradients of f(z) = 5*z where z = f(x) - f(y) w.r.t x is {}".format(fz_2.grad(x)))
# gradient of fz_3 w.r.t x is d5(x^2/(-y))/dx = -5x = -20
print("Gradients of f(z) = 5*z where z = f(x) / f(y) w.r.t x is {}".format(fz_3.grad(x)))

# illustratation of taking two variables as input 
# gradient of fz_1 w.r.t y is d5(x^2-y)/dy = -5
print("Gradients of f(z) = 5*z where z = f(x) + f(y) w.r.t y is {}".format(fz_1.grad(y)))

#part 2
# gradient of f(a) = sin(a) w.r.t a is 1
a.forward()
print("Gradient of f(a) = sin(a) w.r.t a is {}".format(fa.grad(a)))
# gradient of f(b) = arcsin(b) w.r.t b is 1
b.forward()
print("Gradient of f(b) = arcsin(b) w.r.t y is {}".format(fb.grad(b)))
# gradient of f(c) = 1/(1+e^(-c)) + log2(c) + c^(1/2) w.r.t c is 2.14
c.forward()
print("Gradient of f(c) = 1/(1+e^(-c)) + log2(c) + c^(1/2) w.r.t c is {}".format(fc.grad(c)))

Gradient of f(x) = x^2 w.r.t x is 8.0
Gradient of f(y) = -y w.r.t y is -1.0
Gradients of f(z) = 5*z where z = f(x) + f(y) w.r.t x is 40.0
Gradients of f(z) = 5*z where z = f(x) - f(y) w.r.t x is 40.0
Gradients of f(z) = 5*z where z = f(x) / f(y) w.r.t x is -20.0
Gradients of f(z) = 5*z where z = f(x) + f(y) w.r.t y is -5.0
Gradient of f(a) = sin(a) w.r.t a is 1.0
Gradient of f(b) = arcsin(b) w.r.t y is 1.0
Gradient of f(c) = 1/(1+e^(-c)) + log2(c) + c^(1/2) w.r.t c is 2.139306974130445


#### Reverse Mode

Below we show an example of how to access gradients of $f(x)$, $f(y)$, $f(z)$ and $f(a)$, $f(b)$, $f(c)$using `.grad(variable)` method via reverse mode.

In [0]:
# accessing gradients
#part 1
# gradient of f(x) = x^2 w.r.t x is 8
fx.backward()
print("Gradient of f(x) = x^2 w.r.t x is {}".format(fx.grad(x)))
# gradient of f(y) = -y w.r.t y is -1
fy.backward()
print("Gradient of f(y) = -y w.r.t y is {}".format(fy.grad(y)))

# gradient of fz_1 w.r.t x is d5(x^2-y)/dx = 10x = 40
fz_1.backward()
print("Gradients of f(z) = 5*z where z = f(x) + f(y) w.r.t x is {}".format(fz_1.grad(x)))
# gradient of fz_2 w.r.t x is d5(x^2+y)/dx = 10x = 40
fz_2.backward()
print("Gradients of f(z) = 5*z where z = f(x) - f(y) w.r.t x is {}".format(fz_2.grad(x)))
# gradient of fz_3 w.r.t x is d5(x^2/(-y))/dx = -5x = -20
fz_3.backward()
print("Gradients of f(z) = 5*z where z = f(x) / f(y) w.r.t x is {}".format(fz_3.grad(x)))

# illustratation of taking two variables as input 
# gradient of fz_1 w.r.t y is d5(x^2-y)/dy = -5
print("Gradients of f(z) = 5*z where z = f(x) + f(y) w.r.t y is {}".format(fz_1.grad(y)))

#part 2
# gradient of f(a) = sin(a) w.r.t a is 1
fa.backward()
print("Gradient of f(a) = sin(a) w.r.t a is {}".format(fa.grad(a)))
# gradient of f(b) = arcsin(b) w.r.t b is 1
fb.backward()
print("Gradient of f(b) = arcsin(b) w.r.t y is {}".format(fb.grad(b)))
# gradient of f(c) = 1/(1+e^(-c)) + log2(c) + c^(1/2) w.r.t c is 2.14
fc.backward()
print("Gradient of f(c) = 1/(1+e^(-c)) + log2(c) + c^(1/2) w.r.t c is {}".format(fc.grad(c)))

Gradient of f(x) = x^2 w.r.t x is 8.0
Gradient of f(y) = -y w.r.t y is -1.0
Gradients of f(z) = 5*z where z = f(x) + f(y) w.r.t x is 40.0
Gradients of f(z) = 5*z where z = f(x) - f(y) w.r.t x is 40.0
Gradients of f(z) = 5*z where z = f(x) / f(y) w.r.t x is -20.0
Gradients of f(z) = 5*z where z = f(x) + f(y) w.r.t y is -5.0
Gradient of f(a) = sin(a) w.r.t a is 1.0
Gradient of f(b) = arcsin(b) w.r.t y is 1.0
Gradient of f(c) = 1/(1+e^(-c)) + log2(c) + c^(1/2) w.r.t c is 2.139306974130445


### Working with Vectors

Our class Var is a wrapper for numpy arrays that can handle both scalars and vectors.

Below we show an example for evaluating the value and the gradients/Jacobian of $f(z) = 2*z$ where $z = f(x,y)$ and  $f(x,y) = x^2+y$ where $x$,$y$ are the vectors:

In [0]:
# creating vector variables x = (1, 2, 3, 4, 5) y = (1, 2, 3, 4, 5)
x = Var([1, 2, 3, 4, 5])
y = Var([1, 2, 3, 4, 5])

# constructing and evaluating f(x,y) and f(z)
fxy = x**2 + y
fz = 2*fxy

# accessing the evaluated values
print("Evaluating f(x,y) = x^2 + y : {}".format(fxy.val))
print("Evaluating f(z) = 2*z : {}".format(fz.val))

# Jacobian of f(x,y) w.r.t x
x.forward()
print("Jacobian of f(x,y) w.r.t. x:\n{}".format(fxy.grad(x)))
# Jacobian of f(z) w.r.t x
print("Jacobian of f(z) w.r.t. x:\n{}".format(fz.grad(x)))
# Jacobian of f(z) w.r.t y
y.forward()
print("Jacobian of f(z) w.r.t. y:\n{}".format(fz.grad(y)))

Evaluating f(x,y) = x^2 + y : [ 2.  6. 12. 20. 30.]
Evaluating f(z) = 2*z : [ 4. 12. 24. 40. 60.]
Jacobian of f(x,y) w.r.t. x:
[ 2.  4.  6.  8. 10.]
Jacobian of f(z) w.r.t. x:
[ 4.  8. 12. 16. 20.]
Jacobian of f(z) w.r.t. y:
[2. 2. 2. 2. 2.]


### Easy-To-Use Newton's Method with lazydiff

We show how we can use Newton's Method to solve for root of a question using lazzydiff

In this case, we explore this using $f(x) = x^2 - 2$, which has a root at $x=\sqrt2$

In [0]:
def Newton_Method(f, x0, maxiter=1000):
    """ returns the root for the given function
        using Newtons Method
        
        this function supports only Scalar
        you can modify this function to support Vector
        
        ===INPUT===
        f: function written in ops or built_in arithmetic functions
        x0: initial point
        maxiter: maximum number of iterations
    """
    x = x0
    for _ in range(maxiter):
        if abs(f(x)).val < 1e-8:
            return x
        fx = f(x)
        fx.backward()
        newx = x - fx/fx.grad(x)
        if (abs(newx-x).val < 1e-8):
            return newx
        x = newx

In [0]:
# the function x^2-2 for which we want to find the root of
fx = lambda x: x**2-2
ans = Newton_Method(fx, Var(1))
print("The root found by Newton's Method using autodiff is: {}".format(ans.val))
print("The difference between this root and the real root 2**0.5 is : {}".format(ans.val-2**0.5))

The root found by Newton's Method using autodiff is: 1.4142135623746899
The difference between this root and the real root 2**0.5 is : 1.5947243525715749e-12


# Background
 AD exploits the fact that every function, no matter how complicated, is a sequence of elementary arithmetic operations (addition, subtraction, multiplication, division, etc.) and elementary functions (such as $\sin$, $\cos$, $\exp$, $\log$, etc.). By applying the chain rule repeatedly to these operations, derivatives of arbitrary order can be computed automatically, accurately to working precision, and using at most a small constant factor more arithmetic operations than the original program.
 * **Chain Rule**  
   - AD relies heavily on the chain rule from calculus, which says that $$\frac{d}{dx} f(g(x)) = \frac{df}{dg} \frac{dg}{dx} $$ For a more complex composition $$y = f(g(h(x))) = f(g(h(x_1))) = f(g(x_2)) = f(x_3) = x_4$$ the chain rule gives   
    $$\frac{dy}{dx} = \frac{dy}{dx_3} \frac{dx_3}{dx_2} \frac{dx_2}{dx_1}\frac{dx_1}{dx}$$  
    
 * **Forward Accumulation**:  
 We can think of taking derivatives in AD as building a graph representing evaluation of a function, and using the chain rule to propagate derivative values from input variables to the final function, where each node represents an elementary operation or elementary function.
 
 Below is a sample such graph structure resulting from evaluating the function $$f(x_1,x_2) = \sin(x_1)  + x_1 x_2$$
 
 ![](https://upload.wikimedia.org/wikipedia/commons/a/a4/ForwardAccumulationAutomaticDifferentiation.png)
 [image source](https://upload.wikimedia.org/wikipedia/commons/a/a4/ForwardAccumulationAutomaticDifferentiation.png)
 
 We see that the input values $x_1,x_2$ and their derivatives $\dot{w_1}, \dot{w_2}$ propagate upward and are used in computations for the derivatives $\dot{w_3}, \dot{w_4}, \dot{w_5}$ at each node on the way up to the final function $f(x_1,x_2) = w_5$. 
 
<!--  ![alt text](https://upload.wikimedia.org/wikipedia/commons/a/a0/ReverseaccumulationAD.png =400x200)   \\ -->
 
 * **Reverse Accumulation**:  
 In reverse accumulation AD, one first fixes the dependent variable to be differentiated and computes the derivative with respect to each sub-expression recursively. It traverses the chain rule from outside to inside.
 
 Below is the same graph structure for $f(x_1, x_2)$, but with reverse propagation notated instead.
 
 ![](https://upload.wikimedia.org/wikipedia/commons/a/a0/ReverseaccumulationAD.png)
 [image source](https://upload.wikimedia.org/wikipedia/commons/a/a0/ReverseaccumulationAD.png)
 
 * **AD in Linear Regression**:  
 Linear regression model is modeling a dependent variable linearly dependent on some set of independent variables in a noisy environment. $$y^{(i)} = \boldsymbol{\theta}^{T}\boldsymbol{x}^{(i)} + \epsilon^{(i)} $$
We can write the likelihood function given all the observations as: $$ \mathcal{L}(\boldsymbol{\theta}; X, \boldsymbol{y}) =  \prod_{i = 1}^n \frac{1}{\sqrt{2\pi}\sigma}\exp\big(\frac{-(y^{(i)} - \boldsymbol{\theta}^{T}\boldsymbol{x}^{(i)})^2}{2\sigma^2}\big)  $$
In order to find the best fitting parameters $\boldsymbol{\theta}$ we therefore need to maximize this function with respect to $\boldsymbol{\theta}$. The standard approach is to maximize the log likelihood which, since log is monotonic, will give the same result. Hence maximizing the likelihood is the same as minimizing the estimate of the variance which is also known as loss function.  $$\mathcal{J}(\boldsymbol{\theta}) = \sum_{i=1}^n (y^{(i)} - \boldsymbol{\theta}^{T}\boldsymbol{x}^{(i)})^2$$ 
In order to mininize the cost function, we use the method of steepest descent$$   \boldsymbol{\theta}^{i+1} = \boldsymbol{\theta}^{i} - \alpha \nabla\mathcal{J}(\boldsymbol{\theta}) $$
where we can use automatic differentiation.

# Software Organization

## Directory Structure
We keep the old implementation in one of
the branches and not test for it, since we made significant changes to our old implementation and do not use the old one anymore. Our new
implementation is all in 'code' branch.

Our directory structure looks like the following:
* `lazydiff/`
     * `__init__.py`
     * `vars.py`
     * `ops.py`
     * `regression.py`
     * `tests/`
       * `__init__.py`
       * `test_scalar.py`
       * `test_ops.py`
       * `test_vector.py`
       * `test_vector_ops.py`
       * `test_regression.py`
* `demo/`
 * `Demo.mov`
 * `demo_code.ipynb`
 * `new_implementation.png`
 * `old_implementation.png`
 * `presentation.ipynb`
 * `presentation.slides.html`
* `docs/`
 * `milestone1.ipynb`
 * `milestone2.ipynb`
 * `documentation.ipynb`
* `README.md`
* `LICENSE`
* `setup.py`
* `requirements.txt`
* `.travis.yml`
* `.coveragerc`
* `setup.cfg`

## Basic Modules
All the modules related to autodiff are placed inside the `lazydiff` folder. Files outside this folder are miscellaneous files for setups, Github or test coverages.

The key modules included are `vars.py`, which contains our base Var class as a wrapper of numpy arrays that can handle both scalars and vectors and built-in functions for which we may want to compute, and `ops.py`, which contains methods that allow us to evaluate elementary functions such as $\sin$, $\cos$, etc.

In addition, a module `regression.py` is included as a demonstration of our autodiff capabilities.

## Tests
The test suite live in the folder "tests" in the directory structure above. We use TravicsCI for continuous integration and Coveralls for test coverage.

## Install Package
The packaged library is distributed through Github which can be installed using the pip command and the tutorial is available above in the `How to Install` section.

# Implementation

## Core Data Structures

The core of this library relies on the idea of variables, which store a float for the value it represents; a dictionary of previosly computed gradient values given variable reference as key; a dictionary of the variable's parents' gradients given parent variable reference as key and a dictionary of the gradients of variable's children given children variable reference as key.

Parents dictionary is used in forward mode and children dictionary is used in reverse mode. Also, through the above data structure, we can easily find the gradient with respect to any past variable used without recomputing it each time. Please see the demo for an example of this. 

The numpy based implemnetation can handle both scalars and vectors. All the operations are performed on each entry in a vector.

## Core Classes and Important Attributes

* In module `vars` :
    * We have a class `Var` which contains:
        * Constructor which takes in a value `val`
            * Initializes empty gradient dictionary `grad_val`, which will contain derivatives of this variable with respect to other variables.
            * Initializes dictionary representing gradient calculation parents (dictionary with parent variables as keys and gradients of parents as values). For instance, if we are evaluating the derivative of $\sin x$ at $x = 3$, $$\frac{d}{dx} \sin x \Big|_{x = 3}= \dot{x}|_{x=3} \cos 3$$ so the dictionary of object $\sin x$ would include the the item $\{x: \cos 3\}$.
            * Initializes dictionary representing gradient calculation children (dictionary with child variables as keys and gradients of children as values). For instance, if we are evaluating the derivative of $\sin x$ at $x = 3$, $$\frac{d}{dx} \sin x \Big|_{x = 3}= \dot{x}|_{x=3} \cos 3$$ so the dictionary of object $x$ would include the the item $\{\sin x: \cos 3\}$.
        * Method ```forward``` which visits all descendants of a given variable and then calculates the gradient of each descendant with respect to the given variable, and stores them into `grad_val` dictionary. 
        * Method ```backward``` which visits all ancestors of a given variable and then calculates the gradient of the given variable with respect to each ancestor, and stores them into `grad_val` dictionary. 
        * Method ```grad``` which takes in a variable, and returns a numpy array containing the derivative with respect to that variables. This will only work if the appropriate forward/backward calls have been made. 
        * Methods `__add__`, `__mul__`, etc and comparison operators for overloading basic arithmetic operations.
      
## Elementary functions

* In module `ops`:
    * We have a method for each elementary function (`trig functions`, `inverse trig functions`, `exponentials`, `hyperbolic functions`, `logistic function`, `logarithms` and `square root`, etc.). Each method:
        * Take in a `Var` object 
        * Create a new `Var` object with value updated to reflect execution of elementary function on value of input `Var` object, updated gradient calculation parents and children dictionaries - these compounds functions together
        
    * We have implemented all the basic arithmetic functions from  `numpy` libraries, by directly calling these functions for evaluating the values. Thus, our `ops` library supports basic elementary functions ranging from `abs` to `asin` etc.
    
## Additional Modules
* In module `regression`:
 * We have methods for Mean Square Error (`MSE`) and MSE with regularization (`MSE_regularized`) which are loss functions in linear regression. Each method takes in the dependent,  independent variables, coefficients and intercept. `MSE_regularized` takes in the weight in L-p norm of the coefficients vector as well. For regularization part, we implement Lasso, Ridge and Elastic algorithms. 
 * We also have a `gradient_descent` method which performs one single update step of gradient descent and returns the updated parameters m, b, and loss. It takes in the dependent, independent variables, coefficients, intercept and learning rate. Also, it takes in forward/reverse mode as well to define which mode do we want to use. 
  * `iterative_regression` is a method with a timer decorator performing iterative regression with the given loss function and minizing the loss function with respect to the parameters. We are able to compare the speed of forward and backward due to the timer we used. It takes in the dependent, independent variables, coefficients, intercept, learning rate, epochs, number of iterations for early stopping and forward/reverse mode. 
    
## External Dependencies

The implementations rely on `numpy` for evaluating elementary functions, such as cos, sin, tan, and use numpy arrays to store variable values.

# Future

## Option for higher-order derivatives (Hessians and beyond)

We are thinking of supporting higher-order derivatives, such as Hessians. This can be done by further extending the Vector class with additional methods named as `.Hessian(variables)`. The main challenges of this extension is to figure out how to utilize `.grad(variables)` method to effectively get Hessians or beyond in a recursive manner to prevent reusing duplicate codes. Another challenge is how to do thorough testings of different operations on Hessian and beyond, which can be quite labor intensive.

## More application modules

We currently have a module `regression` that allows a user to easily use lazydiff for some basic regression tasks, but it would be nice if we could add more modules to support solving other optimization problems with lazydiff as well.